In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from aux_functions import *
from sklearn.externals import joblib
from scipy.ndimage.measurements import label
from collections import deque
from moviepy.editor import VideoFileClip, ImageSequenceClip
from IPython.display import HTML
%matplotlib inline

## Region of Interest

I'll start by defining what area of the images we are going to look for cars in. This will be a horizontal strip where y pixel coordinate would vary between 400 and 600.

In [2]:
ystart = 400
ystop = 600

We need to classify images so I need to import the classifier that has been previously trained.

In [3]:
classifier = joblib.load('final_svm.pkl')
scaler = joblib.load('final_scaler.pkl')

I'll set in here the scales to use in this case. All other parameters are going to be taken with default values. This is accomplish by default values passed in [aux_functions.py](./aux_functions.py). Those values have been found during the work explained in the other notebooks. I believe this makes it more readable.

In [4]:
scales = [1, 1.5, 2]
filtered_boxes = deque()
nframe_filter = 5

In [5]:
# helper function to make ploting shorter
def toRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

Processing of the video is quite straightforward in that it follows what has been done for images. The main difference is the use of a fifo queue that keep track of detected boxes. This queue is implemented in the variable filtered_boxes using python's deque.

Each frame is pass to process_video. In that function, I search for cars in the frame. The list of boxes were the classifier has detected car are accumulated in the fifo queue. Using this queue to calculate the heat map allows to take into consideration previous frames. This help with eliminating false positives.

Another measure that proved very effective in reducing false positives was using the decision function of the classifier instead of the prediction. Thus, in aux_functions.py I set a threshold in the form: test_prediction = classifier.decision_function(test_features) with test_prediction > 0.9.

In [6]:
def process_video(img):
    # my code assumes image is read in BGR as it is by opencv. Image from moviepy come as RGB
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    car_boxes = []
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    # Take several scales and detect boxes. Add all boxes to a list
    # This uses global variables set in this notebook and default values set in aux_functions.py
    for scal in scales:
        car_boxes += find_cars(img, ystart, ystop, scal, classifier, scaler)
    filtered_boxes.appendleft(car_boxes)
    if len(filtered_boxes) > nframe_filter:
        filtered_boxes.pop()
    for boxes in filtered_boxes:
        # Add heat to each box in box list
        heat = add_heat(heat,boxes)
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat, nframe_filter)
    
    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(img), labels)
    return cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)

In [8]:
video_output = './test_video_processed.mp4'
clip = VideoFileClip('./test_video.mp4')
clip_processed = clip.fl_image(process_video)
%time clip_processed.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video ./test_video_processed.mp4
[MoviePy] Writing video ./test_video_processed.mp4


 97%|█████████▋| 38/39 [00:44<00:01,  1.26s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./test_video_processed.mp4 

CPU times: user 5min 2s, sys: 9.28 s, total: 5min 12s
Wall time: 45.3 s


In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

In [11]:
video_output = './project_video_processed.mp4'
clip = VideoFileClip('./project_video.mp4')
clip_processed = clip.fl_image(process_video)
%time clip_processed.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video ./project_video_processed.mp4
[MoviePy] Writing video ./project_video_processed.mp4


100%|█████████▉| 1260/1261 [56:31<00:02,  2.58s/it] 


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_processed.mp4 

CPU times: user 4h 50min 57s, sys: 8min 16s, total: 4h 59min 13s
Wall time: 56min 32s


Results are fairly good. I personally think that my pipeline lacks robustness as it has proven to be quite sensible to tweaking values of scales, thresholds and so on.

Another issue seems to be the time is takes to process each frame. To identify bottlenecks, I have run a very simple profiling. As was expected, feature extraction takes quite some time for each frame. Some of it is repeated several times. The first thing that jumps to mind is reducing the time taken by color features extraction. An approach similar to what is done with hog features could be greatly beneficial. This could be done by restricting the scales to multiples of a minimum scale. The features could be calculated at that scale and added in form of cell latter, instead of calculating them again at each scale.

## Some Profiling

In [7]:
import cProfile
import pstats
test_image = cv2.imread('test_images/test4.jpg')
cProfile.run('process_video(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))', 'run_stats')
p = pstats.Stats('run_stats')
p.sort_stats('time').print_stats(15)

Thu Mar 23 18:15:22 2017    run_stats

         247588 function calls in 2.193 seconds

   Ordered by: internal time
   List reduced from 84 to 15 due to restriction <15>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.440    0.049    0.810    0.090 /home/alberto/anaconda3/lib/python3.5/site-packages/skimage/feature/_hog.py:8(hog)
     2628    0.396    0.000    0.709    0.000 /home/alberto/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:267(histogram)
        9    0.203    0.023    0.203    0.023 {skimage.feature._hoghistogram.hog_histograms}
      882    0.198    0.000    0.198    0.000 {cvtColor}
    25794    0.164    0.000    0.164    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    10513    0.056    0.000    0.056    0.000 {method 'astype' of 'numpy.ndarray' objects}
     2628    0.053    0.000    0.053    0.000 {built-in method numpy.core.multiarray.bincount}
     2628    0.049    0.000    0.069    0.000 /home/alberto/an